In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib notebook 
plt.rcParams["figure.figsize"] = (3,3)
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp

Suivi de position en mer
======================

Le but de ce projet est de simuler et prévoir en temps réel la position de navires en mer à partir de données GPS. En effet, la plupart de la flotte mondiale est maintenant équippée d'un AIS (Automatic Identification System) 

https://shipping.nato.int/nsc/operations/news/2021/ais-automatic-identification-system-overview

qui communique régulièrement aux côtes et aux satellites la position de chaque bateau. Les données sont en fait disponibles publiquement et en temps réel sur 

https://www.marinetraffic.com/en/ais/home/centerx:-4.5/centery:50.8/zoom:6

Ces communications permettent le suivi des positions et l'identification des navires à la fois aux côtes et à chacun des navires pour éviter les collisions.

Considérons un bateau modélisé par sa position 2D $(x_1,x_2)\in \mathbb{R}^2$ dans le plan de la mer (on néglige la houle), et son orientation $x_3\in \mathbb{R}$
\begin{align*}
\dot{x}_1 &= v(t) \cos(x_3) \\
\dot{x}_2 &= v(t) \sin(x_3) \\
\dot{x}_3 &= w(t)
\end{align*}
où $v$ et $w$ sont les vitesses linéaires et de rotation respectivement. On souhaite estimer la position en temps réel de ce bateau à partir de mesures GPS bruitées sans connaître les signaux $v$ et $w$. Pour cela, nous allons d'abord étudier la simulation numérique des trajectoires de ce bateau, puis apprendre à implémenter un *filtre de Kalman* permettant le filtrage des données GPS grâce à la connaissance d'un modèle approximé. 

Notons que dans ce projet, nous supposons la position GPS disponible continûment (à chaque instant) alors qu'elle est en fait échantillonnée en pratique, et il faut alors utiliser des méthodes discrètes mêlant la prédiction par le modèle et le recalage par les mesures (filtre de Kalman discret, filtre particulaire). Ce sont des algorithmes probabilistes que vous verrez dans le projet numérique de probabilités.


In [ ]:
def f(t,x):
    v = 100
    w = 100
    return np.array([v*np.cos(x[2]),
                     v*np.sin(x[2]),
                     w])

Euler explicite
================

**Question 1** Compléter les fonctions ``solve_euler_explicit`` et ``predict_euler_explicit`` ci-dessous prenant en entrée une fonction $f: \mathbb{R}\times\mathbb{R}^n \to \mathbb{R}^n$ quelconque, une condition initiale $x_0$, un pas de temps $dt$, les temps initiaux et finaux, et renvoyant 

- le vecteur des temps $t^j$ et de la solution $x^j$ du schéma d'Euler explicite appliqué à $\dot{x}=f(t,x)$,

- la valeur finale au temps $t_f$ de la solution du schéma d'Euler explicite appliqué à $\dot{x}=f(t,x)$,

respectivement.

In [ ]:
def solve_euler_explicit(f, x0, dt, t0, tf):
    t, x = [t0], [x0]
    while t[-1]<= tf:
        x_iter = x[-1] + dt*f(t[-1], x[-1])
        x.append(x_iter)
        t.append(t[-1] + dt)
    
    return np.array(t),np.array(x)

In [ ]:
def predict_euler_explicit(f, x0, dt, t0, tf):
    t_list,x_list = solve_euler_explicit(f, x0, dt, t0, tf)
    return 

**Question 2** Utiliser ``solve_euler_explicit`` pour simuler des trajectoires simples et connues de bateau (ligne droite et cercle notamment). Etudier l'erreur numérique du schéma
- lorsque $\Delta t$ tend vers 0 pour un horizon de temps $t_f$ fixé ; 
- à $\Delta t$ fixé, lorsque $t_f$ augmente : arrivez-vous à simuler une trajectoire circulaire en temps long?

Expliquer comment trouver numériquement (sans le faire) l'ordre de convergence du schéma.

In [ ]:
def f_ligne(t,x):
    """modélise une trajectoire rectiligne"""
    v = 100
    w = 0
    return np.array([v*np.cos(x[2]),
                     v*np.sin(x[2]),
                     w])

x0 = np.array([100,100,20])
t_list, x_list = solve_euler_explicit(f_ligne, x0, 1, 0, 10)

plt.figure()
plt.plot(x_list[:,0],x_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')

In [ ]:
# erreur numérique dans le cas de la ligne droite
v1 = 100

def norme(X):
    return np.sqrt(X[0]**2 + X[1]**2 + X[2]**2)

def x1(t):
    return v1*np.cos(x0[2])*t + x0[0]

vec_x1 = np.vectorize(x1)

def x2(t):
    return v1*np.sin(x0[2])*t + x0[1]

vec_x2 = np.vectorize(x2)

def erreur_numérique1(dt):
    Z = solve_euler_explicit(f_ligne, x0, dt, 0, 10)
    X_approché = Z[1]
    X_réel = np.transpose([vec_x1(Z[0]), vec_x2(Z[0]), [x0[2] for t in Z[0]]])
    erreur = [norme(X_approché[i] - X_réel[i]) for i in range(len(Z[0]))]
    return max(erreur)
  
vec_erreur_numérique1 = np.vectorize(erreur_numérique1)

list_t = np.linspace(0.01,10,50)

plt.figure()
plt.plot(list_t, vec_erreur_numérique1(list_t))
plt.xlabel('pas de temps')
plt.ylabel('erreur numérique')
plt.grid(True)
plt.show()

In [ ]:
def f_cercle(t,x):
    """modélise une trajectoire circulaire"""
    v = 100
    w = 100
    return np.array([v*np.cos(x[2]),
                     v*np.sin(x[2]),
                     w])

x0 = np.array([100,100,0])
t_list, x_list = solve_euler_explicit(f_cercle, x0, 1, 0, 200)

plt.figure()
plt.plot(x_list[:,0],x_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')



In [ ]:
# erreur numérique dans le cas du cercle
v=100
w=100

def x1(t):
    return v*(np.sin(w*t + x0[2]) - np.sin(x0[2]))/w + x0[0]

def x2(t):
    return v*(np.cos(x0[2]) - np.cos(w*t + x0[2]))/w + x0[1]

def erreur_numérique2(dt):
    Z = solve_euler_explicit(f_cercle, x0, dt, 0, 10)
    X_approché = Z[1]
    X_réel = np.transpose([vec_x1(Z[0]), vec_x2(Z[0]), [x0[2] for t in Z[0]]])
    erreur = [norme(X_approché[i] - X_réel[i]) for i in range(len(Z[0]))]
    return max(erreur)
  
vec_erreur_numérique2 = np.vectorize(erreur_numérique2)

plt.figure()
list_t = np.linspace(0.01,10, 50)
plt.plot(list_t, vec_erreur_numérique2(list_t))
plt.xlabel('pas de temps')
plt.ylabel('erreur numérique')
plt.grid(True)
plt.show()

Erreur numérique
- à $t_f$ fixé
- à $\Delta t$ fixé

Trouver ordre de convergence

Modèle d'estimation
====================

Notre fonction de prédiction demande un pas de temps suffisamment petit et surtout nécessite de connaitre les vitesses $v$ et $w$ du bateau. On souhaite être capable de suivre la trajectoire du bateau sans connaitre ces quantités, en exploitant des mesures de position intermittentes obtenues par GPS. On suppose pour cela que les vitesses $v$ et $w$ varient lentement et on adopte donc le modèle étendu (approximé)
\begin{align}
\dot{x}_1 &= x_4 \cos(x_3) \\
\dot{x}_2 &= x_4 \sin(x_3) \\
\dot{x}_3 &= x_5 \\
\dot{x}_4 &= 0 \\
\dot{x}_5 &= 0 
\end{align}

In [ ]:
def f_ext(t,x):
    return np.array([x[3]*np.cos(x[2]),x[3]*np.sin(x[2]),x[4],0,0])

On suppose dans ce projet que l'on dispose d'une mesure GPS de position $y=(x_1,x_2)$ à chaque instant et que l'on souhaite estimer la position $(x_1,x_2)$, l'angle $x_3$ et les vitesses $(x_4,x_5)$ en temps réel. Estimer la position déjà connue par le GPS peut paraître inutile mais la mesure est souvent bruitée et l'information du modèle d'évolution peut permettre de réduire ou "filtrer" ce bruit.

**Question 3** Justifier que le système théorique donné par ``f_ext`` et $y$ est *observable*, c'est-à-dire que la connaissance de $t\mapsto y(t) = (x_1(t),x_2(t))$ détermine de manière unique tout l'état $(x_1,x_2,x_3,x_4,x_5)$. 


$ x_4 = \sqrt{\dot{x_1}^2 + \dot{x_2}^2} \\
 x_3 = \arctan(\frac{\dot{x_2}}{\dot{x_1}}) \\
 x_5 = \dot{x_3} $

Est-ce une bonne idée de calculer analytiquement $x_3,x_4,x_5$ à partir de $y$ ?

On souhaite donc écrire un algorithme, appelé *observateur* ou *filtre*, qui prenne en entrée $y(t)$ et qui construise une estimée $\hat{x}(t)$ de $x(t)$ telle que $\hat{x}$ converge vers $x$ asymptotiquement.

Pour cela, il faut prendre en compte que le modèle donné par ``f_ext`` et $y$ n'est pas exact car il y a 2 sources d'incertitudes : les erreurs de mesure du GPS, et les accélérations $\dot{v}$ et $\dot{w}$ qui ne sont pas nulles en réalité. Le système réel est donc plutôt
\begin{align*}
\dot{x}_1 &= x_4 \cos(x_3) \\
\dot{x}_2 &= x_4 \sin(x_3) \\
\dot{x}_3 &= x_5 \hspace{5em} , \hspace{5em} y_m = (x_1 ,x_2) + \delta_y(t) \\
\dot{x}_4 &= \delta_v(t) \\
\dot{x}_5 &= \delta_w(t)
\end{align*}
avec $\delta_v\in \mathbb{R}$, $\delta_w\in \mathbb{R}$, $\delta_y\in \mathbb{R}^2$, des entrées *faibles* mais inconnues. L'observateur doit donc être *robuste* aux erreurs de modèles et de mesure.

Etude théorique du filtre de Kalman étendu déterministe
==============

Considérons un système dynamique décrit par
$$
\dot{x} = f(t,x) + \delta_f(t) \quad , \quad y_m = h(t,x)+ \delta_h(t) 
$$
où $x\in \mathbb{R}^n$ est l'état du système, $y_m\in \mathbb{R}^p$ est la mesure connue, et $\delta_f,\delta_h$ sont des incertitudes inconnues que l'on suppose bornées. 

Etant donné la mesure $t\mapsto y_m(t)$, on s'intéresse alors à implémenter un *filtre de Kalman* donné par
\begin{align}
\dot{\hat{x}} & = f(t,\hat{x})+ P\frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} (y_m(t)-h(t,\hat{x})) 
\\
\dot{P} & = \lambda P + \frac{\partial f}{\partial x}(t,\hat{x}) P + P\frac{\partial f}{\partial x}(t,\hat{x})^\top + Q - P \frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} \frac{\partial h}{\partial x}(t,\hat{x}) P
\end{align}
où $\hat{x}\in \mathbb{R}^n$, $P\in \mathbb{R}^{n\times n}$ constitue l'état du filtre, et où les paramètres $\lambda\in \mathbb{R}$, $Q\in \mathbb{R}^{n\times n}$, $R\in \mathbb{R}^{p\times p}$ sont à choisir tels que
- $\lambda \geq 0$ et $Q$ est symétrique, à valeurs propres positives (matrice positive),
- $R$ est symétrique, à valeurs propres strictement positives (matrice définie positive),
- $\lambda > 0$ ou bien $Q$ est à valeurs propres strictement positives.

Cet algorithme compare à chaque instant les mesures aux estimées et corrige ainsi ces estimées dynamiquement, en temps réel. En effet, $x(0)$ étant mal connue, le modèle étant imparfait, et les erreurs numériques s'accumulant, la simulation sans ces corrections serait impossible. On va donc montrer que modulo les erreurs de modèle et de mesure, c'est algorithme converge localement, c'est-à-dire, $\hat{x}$ tend vers $x$ si l'erreur initiale $\hat{x}(0)-x(0)$ est suffisamment faible.

Pour cela, on admet ici que 

- si $P(0)$ est symétrique à valeurs propres strictement positives, alors $P(t)$ le reste pour tout $t\geq 0$. 
- De plus, on admet que le long des trajectoires, il existe $\rho_1>0$ et $\rho_2>0$ tels que 
$$
\rho_1 I_n \leq P^{-1}(t) \leq \rho_2 I_n \quad \forall t\geq 0 \, (0.5)
$$
où $I_n$ est la matrice identité de dimension $n$. En particulier, $P$ ne tend pas à devenir singulière, ni diverger. Cette dernière hypothèse est cruciale dans la théorie du filtre de Kalman et se justifie sous des hypothèses d'*observabilité* dans le cas des systèmes linéaires. Mais l'étude de cette hypothèse nous amènerait bien trop loin pour ce projet... (voir remarque ci-dessous)

En considérant le système complet d'état $(x,\hat{x},P)$ avec $P$ inversible, posons
$$
V(x,\hat{x},P) = (\hat{x}-x)^\top P^{-1} (\hat{x}-x) \ .
$$
et 
$$
g(t) = V(x(t),\hat{x}(t),P(t))
$$

**Question 4** On suppose que les dérivées partielles de $f$ et $h$ par rapport à $x$ au premier et second ordre sont globalement bornées. $(1)$ 

En déduire qu'étant donnés $Q,R,\lambda, \rho_1,\rho_2$, 

il existe $\kappa_f\geq 0$, $\kappa_h\geq 0$, $k>0$, et $\varepsilon >0$, $\delta\geq 0$, tels que 

pour tout $(t,x,\hat{x})\in \mathbb{R}^+ \times \mathbb{R}^n \times \mathbb{R}^n$,
\begin{align}
\left\|f(t,x)-f(t,\hat{x})-\frac{\partial f}{\partial x}(t,\hat{x}) (x-\hat{x})\right\| &\leq \kappa_f \|x-\hat{x}\|^2 \, (2)\\
\left\|h(t,x)-h(t,\hat{x})-\frac{\partial h}{\partial x}(t,\hat{x}) (x-\hat{x})\right\| &\leq \kappa_h \|x-\hat{x}\|^2 \, (3)
\end{align}
et 
$$
\frac{d}{dt}V(x(t),\hat{x}(t),P(t)) \leq - k V(x(t),\hat{x}(t),P(t)) + \delta \, (4)
$$
le long des trajectoires vérifiant $\|\hat{x}(t)-x(t) \|<\varepsilon$. 

En déduire qu'il existe $k'>0,\rho'>0,\delta'\geq 0$ tels que 

si $\|\hat{x}(0)-x(0)\|$, $\delta_f$ et $\delta_h$ sont suffisamment faibles, 

alors
$$
\|\hat{x}(t)-x(t) \| \leq \rho' e^{-k' t} \|\hat{x}(0)-x(0) \|  + \delta' \qquad \forall t\geq 0, \, (5)
$$
avec $\delta'=0$ en l'absence d'incertitudes, i.e., si $\delta_f=0$ et $\delta_h=0$.

*Indice* : 
- Montrer que $\frac{d}{dt}P^{-1}(t)=-P^{-1}(t) \left(\frac{d}{dt}P(t) \right) P^{-1}(t) \, (6) $. 

- On rappelle aussi que par Cauchy-Schwarz, pour tout $(x,y)\in \mathbb{R}^n\times \mathbb{R}^n$ et pour toute matrice symétrique $P$ à valeurs propres strictement positives (matrice définie positive), on a $x^\top P y \leq \sqrt{x^\top P x} \sqrt{y^\top P y}$. 

- On rappelle aussi l'inégalité de Young :  pour tout $(a,b) \in \mathbb{R}\times \mathbb{R}$ et pour tout $\gamma>0$, $2 ab \leq \gamma a^2 +\frac{1}{\gamma} b^2$.

Préliminaires :

Notons $||\cdot||$ la norme infinie sur $\mathbb{R}^n$ (avec la base canonique $(e_i)_i$), $||\cdot||_{\infty}$ celle des fonctions continues de $\mathbb{R}^n$ dans $\mathbb{R}^n$ et $|||\cdot|||$ la norme subordonnée à la première ($|||A||| \hat{=} sup_{||x||=1} \|A\cdot x\|$, vérifiant $|||AB||| \leq |||A||| \, |||B|||$).

$f$ est deux fois continûment différentiable

par (1), il existe $M$ tel que pour $t \geq 0$, pour $1 \leq i,j \leq n$,  
$$\left\|\frac{\partial f}{\partial x_i}(t,\cdot)\right\|_{\infty} \leq M \\
\left\|\frac{\partial ^2 f}{\partial x_ix_j}(t,\cdot)\right\|_{\infty} \leq M$$

Pour $ t,x \in \mathbb{R} \times \mathbb{R}^n$, en raisonnant sur les coordonnées du gradient et de la hessienne de $f_i$ en base canonique de $\mathbb{R}^n$ en $(t,x)$ on a alors l'existence de leurs normes subordonnées, qui sont $\leq M$ (c'est aussi vrai pour les différentielles partielles et les transposées de ces applications).

En effet, on détaille le cas où $f$ est à valeur réelle :

on a $\forall t,x,i \in \mathbb{R} \times \mathbb{R}^n \times {1,...,n}, \left| \frac{\partial f}{\partial x_i}(t,x) \right| \leq M$

donc pour $y \in \mathbb{R}$ unitaire et $t,x$, 

$$\left\|\frac{\partial f}{\partial x}(t,x)\cdot y\right\| 
= \left\|\sum_i \frac{\partial f}{\partial x_i}(t,x)y_ie_i\right\| \\
= \left|\frac{\partial f}{\partial x_a}(t,x) y_a\right| 
= \left|\frac{\partial f}{\partial x_a}(t,x)\right| \left|y_a\right| 
\leq M \, 1$$

où $1 \leq a \leq n$ est l'indice du coefficient de module maximal

et donc $|||\frac{\partial f}{\partial x}||| \leq M$

Preuve de (2):

Notons $f_i$ la i-ème fonction composante de $f$ selon la base canonique de $\mathbb{R}^n$ aussi deux fois continûment différentiable

Notons $H_i(t,x)$ la hessienne de $f_i$ en $x$

Pour $1 \leq i \leq n \, , \, t \geq 0 \, , \, (x,\hat{x}) \in \mathbb{R}^n \times \mathbb{R}^n$ on applique Taylor-reste intégral à $f_i(t,\cdot)$ entre $x$ et $\hat{x}$, on a 

$$f_i(t,\hat{x}) - f_i(t,x) - \frac{\partial f_i}{\partial x}(t,\hat{x})\cdot(x - \hat{x}) 
= \int_{0}^{1}  < H_i(t, x + (x - \hat{x}) u) \cdot(x - \hat{x}),
(x - \hat{x}) > (1-u) du $$

par CS, $<A\cdot x,x> \leq \|A\cdot x\|\|x\| \leq |||A||| \, \|x\|^2$

donc par inégalité triangulaire intégrale 

$$ \left| f_i(t,\hat{x}) - f_i(t,x) - \frac{\partial f_i}{\partial x}(t,\hat{x})\cdot(x - \hat{x})\right|
\leq \int_{0}^{1}  M \|x - \hat{x}\|^2(1-u) du 
= \frac{M}{2} \|x - \hat{x}\|^2
$$

donc en passant à $||\cdot||$ et en prenant $\kappa_f = \frac{M}{2}$, 

$$\left\|f(t,x)-f(t,\hat{x})-\frac{\partial f}{\partial x}(t,\hat{x}) \cdot (x-\hat{x})\right\| \leq \kappa_f \|x-\hat{x}\|^2 \, (2)$$

On fait pareil pour (3)


preuve indice
$ \textbf{P}(t)\textbf{P}^{-1}(t) = \textbf{I}_n \\
  \dot{\textbf{P}}(t)\textbf{P}^{-1}(t) + \textbf{P}(t)\dot{\textbf{P}^{-1}}(t) = 0 \\
   \dot{\textbf{P}^{-1}}(t) = -\textbf{P}^{-1}(t)\dot{\textbf{P}}(t)\textbf{P}^{-1}(t) $


Preuve de (4):

On rappelle $$ g(t) = V(x(t),\hat{x}(t),P(t)) $$

Notons $q = \hat{x}-x$

$$g(t) = q^TP^{-1}q = <q,P^{-1}q>$$

bilinéarité produit matriciel  
$$\forall t, \dot{g}(t) = \dot{q}^{T}P^{-1}q + q^{T}(\dot{P^{-1}})q  + q^{T}P^{-1}\dot{q}$$

soit $k>0$, CS

$$\dot{g}(t) + kg(t) \leq 2\sqrt{\dot{q}^{T}P^{-1}\dot{q}}\sqrt{q^{T}P^{-1}q} + q^{T}(\dot{P^{-1}})q + kq^{T}P^{-1}q$$

soit $\gamma>0$, young et indice 1

$$\dot{g}(t) + kg(t) \leq \gamma\dot{q}^{T}P^{-1}\dot{q} + \frac{1}{\gamma}q^{T}P^{-1}q + q^{T}\left(-P^{-1}(t) \dot{P(t)} P^{-1}(t)\right)q + kq^{T}P^{-1}q$$

$\rho_2I_n - P^{-1} \geq 0$ donc $\rho_1||q||^2 \leq q^{T}P^{-1}q \leq \rho_2||q||^2$ donc 

$$\dot{g}(t) + kg(t) \leq \gamma \rho_2||\dot{q}||^2 + q^{T}(\dot{P^{-1}})q + \left(\frac{1}{\gamma} + k \right)\rho_2||q||^2 \, (E)$$

or

$$\dot{q} = \dot{\hat{x}}-\dot{x} 
= f(t,\hat{x}) - f(t,x) + P\frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} (h(t,x) -h(t,\hat{x}) + \delta_h(t) ) - \delta_f(t) $$

$$\dot{P} = \lambda P + \frac{\partial f}{\partial x}(t,\hat{x}) P + P\frac{\partial f}{\partial x}(t,\hat{x})^\top + Q - P \frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} \frac{\partial h}{\partial x}(t,\hat{x}) P$$

$$\dot{P^{-1}} = -P^{-1}(t)(\dot{P})P^{-1}(t) \\  
= - \left( \lambda P^{-1} + P^{-1}\frac{\partial f}{\partial x}(t,\hat{x}) + \frac{\partial f}{\partial x}(t,\hat{x})^\top P^{-1} + P^{-1}QP^{-1} - \frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} \frac{\partial h}{\partial x}(t,\hat{x}) \right)$$

donc

$$ q^T\dot{P^{-1}}q = -q^T\lambda P^{-1}q - q^T \left(P^{-1}\frac{\partial f}{\partial x}(t,\hat{x})q + q^T\frac{\partial f}{\partial x}(t,\hat{x})^\top P^{-1} \right)q - q^TP^{-1}QP^{-1}q + q^T\frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} \frac{\partial h}{\partial x}(t,\hat{x})q \\
\leq - \lambda \rho_1 \|q\|^2 - 2<P^{-1}q,\frac{\partial f}{\partial x}(t,\hat{x})q> + 
\|\frac{\partial h}{\partial x}(t,\hat{x})q\| \|R^{-1} \frac{\partial h}{\partial x}(t,\hat{x})q\|
$$

car 
- $Q \geq 0 $ donc $(P^{-1}q)^TQ(P^{-1}q) \leq 0$
- $R > 0$ donc $R^{-1} > 0$ donc décomposant $R^{-1}x$ sur une base orthonormée de vecteurs propres, on obtient $|||R^{-1}||| \leq \mu$ où $\mu$ est la plus grande valeur propre de $R^{-1}$

donc par CS et existence et sous multiplicité des normes subordonnées : 
$$ q^T\dot{P^{-1}}q \leq 
- \lambda \rho_1 \|q\|^2 + 2\|P^{-1}q\|\|\frac{\partial f}{\partial x}(t,\hat{x})q\| + |||R^{-1}||| \, ||| \frac{\partial h}{\partial x}(t,\hat{x})|||^2 \, \|q\|^2 \\
\leq - \lambda \rho_1 \|q\|^2 + 2|||P^{-1}||| \, \|q\| \, |||\frac{\partial f}{\partial x}(t,\hat{x})||| \, \|q\| + |||R^{-1}||| \, M^2 \, \|q\|^2 \\
\leq (- \lambda \rho_1 + 2M^2 + |||R^{-1}||| \, M^2) \|q\|^2 $$

et

$$\|\dot{q}\| 
\leq \|f(t,\hat{x}) - f(t,x)\| + \|P\frac{\partial h}{\partial x}(t,\hat{x})^\top R^{-1} (h(t,x) -h(t,\hat{x}) + \delta_h(t) )\| + \|\delta_f(t)\| \\
\leq M\|q\| + |||P(t)|||\,|||\frac{\partial h}{\partial x}(t,\hat{x})^\top|||\,|||R^{-1}|||\,
\|h(t,x) -h(t,\hat{x}) + \delta_h(t)\| + \|\delta_f(t)\| \\
\leq \left(M + \frac{1}{\rho_1}\,M\,|||R^{-1}|||\,L \right)\left(M\|q\| + \eta\right) + \eta' \\
\leq \left(1 + \frac{1}{\rho_1}\,|||R^{-1}|||\,L \right)M\left(M\|q\| + \eta\right) + \eta' $$

car
- $\|f(t,\hat{x}) - f(t,x)\| \leq M\|q\|$
- $\delta_f(t), \, \delta_h(t)$ sont majorés
- $\rho_1P \leq I_n$

Enfin $(E)$ devient :
$$\dot{g}(t) + kg(t) \leq \gamma \rho_2\left(\left(1 + \frac{1}{\rho_1}\,|||R^{-1}|||\,L \right)M\left(M\|q\| + \eta\right) + \eta'\right)^2 \\
+ \left(- \lambda \rho_1 + 2M^2 + |||R^{-1}||| \, M^2 \right) \|q\|^2
+ \left(\frac{1}{\gamma} + k \right)\rho_2||q||^2 \\
\leq \gamma \rho_2(\alpha\|q\|^2 + \beta\|q\| + \kappa) \\
+ \left(- \lambda \rho_1 + 2M^2 + |||R^{-1}||| \, M^2 + \frac{1}{\gamma} + k \right)\rho_2||q||^2 $$

où $\alpha, \beta, \kappa$ sont des constantes positives
on prend $\gamma = 1, k = \lambda\rho_1 > 0$ de sorte à ce que le deuxième terme soit $>0$

on pose alors 
$$\delta =\rho_2(\alpha\|q\|^2 + \beta\|q\| + \kappa)
+ \left(- \lambda \rho_1 + 2M^2 + |||R^{-1}||| \, M^2 + 1 + k \right)\rho_2\varepsilon > 0$$

preuve de (5):

on pose pour $t \geq 0$ : $$ r(t) \hat{=} \dot{g(t)} + kg(t) \leq \delta \, (4)$$

on applique la méthode de la variation de la constante à l'équation différentielle obtenue :
- solution système homogène : $g_h(t) = e^{-kt}$
- changement de fonction inconnue : $g = g_h \, u$
- $u$ vérifie alors $r = g_h \, \dot{u}$ ie $\dot{u}(t) = r(t)e^{kt}$ donc 
$$u(t) = \int_0^tr(s)e^{ks}ds + u(0)$$
- d'où $$<q(t),P^{-1}q(t)> = g(t) = e^{-kt}\left(\int_0^tr(s)e^{ks}ds + u(0)\right)$$
- on encadre grâce à $(0.5)$ et par positivité des fonctions  
$$\|q(t)\|^2\rho_1 = <q(t),\rho_1I_nq(t)>\leq <q(t),P^{-1}q(t)> \leq <q(t),\rho_2I_nq(t)> = \rho_2\|q(t)\|^2 \\
\|q(t)\|^2\rho_1 \leq e^{-kt}\left(\int_0^tr(s)e^{ks}ds +  \frac{g(0)}{g_h(0)} \right) \\
\|q(t)\|^2\leq \frac{e^{-kt}}{\rho_1 }\left(\delta \int_0^te^{ks}ds + \rho_2\|q(0)\|^2 \right) \\
\|q(t)\|^2\leq \frac{e^{-kt}}{\rho_1 }\left(\delta \left[\frac{e^{kt} - 1}{k}\right] + \rho_2\|q(0)\|^2 \right) \\
\|q(t)\|^2\leq \frac{\delta}{k \rho_1 }\left(1- e^{-kt}\right) + \frac{e^{-kt}\rho_2}{\rho_1 }\|q(0)\|^2 \\ 
\|q(t)\|^2\leq \frac{\delta}{k \rho_1 } + \frac{e^{-kt}\rho_2}{\rho_1 }\|q(0)\|^2 \\
\|q(t)\| \leq \sqrt{\frac{\delta}{k \rho_1 } + \frac{e^{-kt}\rho_2}{\rho_1 }\|q(0)\|^2} \\
\|q(t)\| \leq \sqrt{\frac{\delta}{k \rho_1 }} + e^{\frac{-kt}{2}}\sqrt{\frac{\rho_2}{\rho_1 }}\|q(0)\| \\
\|\hat{x}(t)-x(t) \| \leq \rho' e^{-k' t} \|\hat{x}(0)-x(0) \|  + \delta'
$$

où on pose 
- $\rho' = \sqrt{\frac{\rho_2}{\rho_1 }}$
- $k' = \frac{k}{2}$
- $\delta' = \sqrt{\frac{\delta}{k \rho_1 }}$

**Remarque** Notons ici que l'analyse théorique du filtre de Kalman admet souvent un cercle vicieux: la dynamique de $P$, et donc les paramètres $\rho_1$ et $\rho_2$, dépendent de $\hat{x}$, et on a besoin de $\rho_1$ et $\rho_2$ pour montrer la convergence de $\hat{x}$ vers $x$. Ce problème peut être résolu dans certains cas pour des structures particulières ou sous des hypothèses très fortes d'observabilité uniforme. En fait, le filtre de Kalman a initialement été développé pour les systèmes linéaires, où la dynamique de $P$ est indépendante de $\hat{x}$ et où les paramètres $\rho_1$ et $\rho_2$ sont donc des propriétés *intrinsèques* du système (liés à son observabilité) et indépendants de $\hat{x}$. La convergence de l'estimation découle alors sans cercle vicieux, et est même *globale*, c'est-à-dire sans restriction sur l'erreur initiale $\|\hat{x}(0)-x(0) \|$ (ni sur les incertitudes). Vous pouvez le vérifier en prenant $\kappa_f=0$ et $\kappa_h=0$. Mais vu sa grande practicité, ce filtre a ensuite été *étendu* aux systèmes nonlinéaires comme décrit ci-dessus (d'où son nom, *extended Kalman filter*, "EKF"), et est l'un des filtres les plus implémentés dans l'industrie (dans sa version discrète). Il est utile de connaître cet algorithme tout en ayant conscience de ses limites... Des variantes, telles que le *unscented Kalman filter* dans sa version probabiliste, permettent parfois d'obtenir de meilleurs résultats. Pour des systèmes très non linéaires, il vaut mieux utiliser des observateurs adaptés plus complexes.

Application au suivi du bateau
================

On commence par simuler précisément une trajectoire continue quelconque de bateau, que l'on considèrera exacte. Bien sûr, en pratique cette solution exacte est inconnue, seule la mesure l'est, elle sera utilisée ici seulement pour comparer la solution estimée à la vraie, et évaluer ses performances.

In [ ]:
def f_traj(t,x):
    v = 2.0 
    w = np.sin(t)
    return np.array([v*np.cos(x[2]),v*np.sin(x[2]),w])

In [ ]:
# simulation d'une trajectoire continue à haute précision, que l'on considère exacte. 
# L'option ``dense_output`` de ``solve_ivp`` permet d'obtenir en sortie une fonction ``sol_traj.sol`` qui donne la solution 
# à n'importe quel temps (voir doc) Ceci est très utile lorsque vous ne savez pas d'avance à quel instant vous en aurez besoin
# (comme ici la mesure, voir fonction ``y_traj`` ci-dessous)
x0 = np.array([0.0,0.0,0.0])
t0, tf = 0.0, 20.0
sol_traj = solve_ivp(fun = f_traj, t_span = [t0, tf], y0 = x0, dense_output=True, t_eval = np.arange(t0,tf,0.1), rtol=1e-4)

plt.figure()
plt.plot(sol_traj.y[0],sol_traj.y[1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Trajectoire exacte')

def y_traj(t):
    # on sélectionne les deux premières composantes de la solution "exacte" au temps t : mesure de position exacte
    return sol_traj.sol(t)[:2] 

print(y_traj(10.0),type(y_traj(10.0)))

Puis on définit les paramètres et fonctions nécessaires à la simulation du filtre.

**Question 5** Compléter ci-dessous les fonctions ``jac_f_ext`` et ``jac_h_ext`` donnant les matrices jacobiennes des fonctions de dynamique et mesure théoriques $f_{ext}:\mathbb{R}^5\to \mathbb{R}^5$ and $h_{ext}:\mathbb{R}^5\to \mathbb{R}^2$ données par
$$
f_{ext}(x) = (x_4 \cos(x_3),x_4 \sin(x_3),x_5 ,0,0) \quad , \quad h_{ext} = (x_1,x_2)
$$
Vu que les incertitudes $\delta_v,\delta_w,\delta_y$ sont inconnues, ce sont ces fonctions qui seront utilisées dans le filtre.

# interrogation

cela veut-il dire que le système est $\dot{x} = f_{ext}(x) \, , \, \dot{y_m} = h_{ext}(y)$ ?

In [ ]:
def mat_mul(mat_list):
    """calcule le produit matriciel x[0] @ x[1] @ ... @ x[m]"""
    if mat_list == []:
        return 1 # np.dot effectue le produit par un scalaire
    else:
        a = mat_list.pop()
        return np.dot(mat_mul(mat_list), a)

In [ ]:
# scénario de mesure
std_y = 1.0  # écart-type du bruit de mesure

# paramètres du filtre de Kalman
# n = 5, r = 2

R = np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))


**Question 6** Compléter la fonction ``f_EKF`` pour simuler la dynamique du filtre de Kalman étendu.

# interrogations

je ne comprends pas le lien entre `f_traj` qui est la dérivée de $x$ fonction dans $\mathbb{R}^3$ et `f_ext` qui est celle de $x$ fonction dans $\mathbb{R}^5$

- avec la 1ère, $\dot{x_3} = w = sin(t)$ la vitesse d'orientation dépend du temps
- avec la 2nde, $\dot{x_3} = x_5$


**Question 7** Implémenter le filtre en appliquant votre fonction ``solve_euler_explicit`` à ``f_EKF``. 

Comparer l'estimation obtenue avec la trajectoire exacte pour différentes valeurs de paramètres $(R,Q,\lambda)$ et conditions initiales, d'abord en absence de bruit de mesure, puis en introduisant le bruit. 

Que se passe-t-il lorsque $R$ est pris petit/grand ? 

Justifier qu'on relie ce paramètre à la confiance que l'on a en la mesure. 

Lorsqu'on rajoute le bruit de mesure, expliquer pourquoi l'on dit souvent que le filtre fait un compromis entre bruit de modèle et bruit de mesure.

In [ ]:
# sur la plage de temps [t0,tf], avec un résolution temporelle dt = 1, les coefficients des matrices & vecteurs divergent 
# et deviennent des +-inf puis des nan ce qui lève une erreur
# pour empêcher cela, il faut utiliser un dt <= 0.001


t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element

plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


Le filtre de Kalman étendu continu et déterministe présenté ici a plusieurs défauts :
- la convergence n'est que locale (mais cela vient de la nonlinéarité et on n'y échappe pas, à moins de recourir à des algorithmes très différents et plus complexes),
- il nécessite d'avoir à disposition la mesure GPS en tout temps,
- il ne dit pas comment choisir les paramètres.

En pratique, on implémente plutôt la discrétisation du filtre de Kalman, qui permet de prédire par le modèle (Euler explicite) la position du bateau lorsqu'aucune mesure n'est disponible, puis de "recâler/corriger" l'estimée lorsque la mesure est disponible, en comparant la mesure à l'estimée comme ci-dessus. 

Par ailleurs, le filtre de Kalman admet, dans le contexte linéaire, une interprétation stochastique qui permet de le voir comme le filtre optimal qui minimise la variance de l'erreur d'estimation lorsque les incertitudes de mesures et de modèles sont interprétées comme des bruits blancs gaussiens. C'est en fait ainsi qu'il a été inventé. Les matrices $R$ et $Q$ sont alors directement reliés à la variance des incertitudes, ce qui en fait un filtre très intuitif à régler et ce qui justifie son succès dans l'industrie. Vous verrez donc la suite de ce projet en cours de Probabilités !